In [1]:
import requests, io, astropy

## For handling ordinary astropy Tables
from astropy.table import Table, vstack

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## our stuff
import sys

# Use the NASA_NAVO/astroquery
from navo_utils.registry import Registry

The Registry.query_counts() method takes arguments which we can use to see which keyword values might help us narrow down our search, or possibly give us too MANY results (these are passed to internal function _build_counts_adql):

    field      : keyword field for which to see popular values: "waveband", "publisher", "service_type" currently supported.
    minimum    : A minimum count of occurences for the keyword value to use as a cutoff (optional, defaults to 1)

In [16]:
results = Registry.query_counts('publisher', 1, verbose=True)
results.show_in_notebook()

Registry:  sending query ADQL = select * from (select role_name as publisher, count(role_name) as count_publisher from rr.res_role where base_role = 'publisher'  group by role_name) as count_table where count_publisher >= 1 order by count_publisher desc

Queried: http://vao.stsci.edu/RegTAP/TapService.aspx/sync



idx,publisher,count_publisher
0,CDS,17373
1,NASA/GSFC HEASARC,1047
2,NASA/IPAC Infrared Science Archive,523
3,The GAVO DC team,179
4,Space Telescope Science Institute Archive,106
5,"WFAU, Institute for Astronomy, University of Edinburgh",101
6,SVO CAB,35
7,IA2,35
8,Canadian Astronomy Data Centre,30
9,Sternberg Astronomical Institute Virtual Observatory Project,29


In [45]:
# These are the NAVO publishers.  Some publisher names should be consolidated.
navo_publishers = [
    # HEASARC
    'NASA/GSFC HEASARC',
    'NASA/HEASARC',
    'HEASARC'
    
    # IRSA
    'NASA/IPAC Infrared Science Archive',
    'NASA/IPAC'
    
    # NED
    'The NASA/IPAC Extragalactic Database',
    
    # MAST
    'Space Telescope Science Institute Archive',
    'Space Telescope Science Institute',
    'STScI'   # These are all open sky nodes.  Should be removed?
]

The Registry.query() method takes arguments which we can use to further filter the results (passed to internal function _build_adql):  

    service_type : "conesearch", "simpleimageaccess", "simplespectralaccess", "tableaccess". May be shortened to "cone", "image", "spectr", or "table" or "tap", respectively.
    keyword      : any keyword contained in ivoid, title, or description
    waveband     : waveband string. Multiple options may be comma-delimited i.e. "optical, infrared"
    source       : any substring in ivoid
    publisher    : the name of any publishing organization
    order_by     : what field to order it by, but then you have to know the names, currently
                    ("waveband","short_name","ivoid","res_description","access_url","reference_url","publisher", service_type")
    logic_string : any other string you want to add to the ADQL where clause, should start with " and "

The results are returned by Registry.query() in an astropy table using the conversion function _astropy_table_from_votable_response(). 



## Waveband Allowed Terms and values in (A)
* gamma-ray:	 	less than 0.1
* X-ray:	 	0.1-100
* EUV:	 	100-1000
* UV:	 	1,000-3,000
* Optical:	 	3,000-10,000
* Infrared:	 	10,000-1,000,000
* Millimeter:	 	10^6 - 10^8
* Radio:	 	over 10^8






In [25]:
heasarc_cone_services = Registry.query( service_type='cone', publisher='heasarc')

print(heasarc_cone_services)

          waveband            short_name ...     publisher     service_type
---------------------------- ----------- ... ----------------- ------------
                       radio  PIGSSBOOFD ... NASA/GSFC HEASARC   conesearch
                       radio      Parkes ... NASA/GSFC HEASARC   conesearch
                       radio  ATLASESCPT ... NASA/GSFC HEASARC   conesearch
      infrared#optical#radio   ATLASESID ... NASA/GSFC HEASARC   conesearch
                     optical         NGC ... NASA/GSFC HEASARC   conesearch
      infrared#optical#x-ray  NGC2237CXO ... NASA/GSFC HEASARC   conesearch
                       x-ray     M101XMM ... NASA/GSFC HEASARC   conesearch
                       radio  INTIBISGAL ... NASA/GSFC HEASARC   conesearch
                       x-ray     XBootes ... NASA/GSFC HEASARC   conesearch
      infrared#optical#x-ray  XBOOTESOID ... NASA/GSFC HEASARC   conesearch
                         ...         ... ...               ...          ...
            

In [5]:
heasarc_image_services = Registry.query( service_type='image', publisher='heasarc')

heasarc_image_services.show_in_notebook()

idx,waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
0,infrared,GOODSHerschel1,ivo://nasa.heasarc/skyview/goodsherschel1,"GOODS-Herschel is an open time key program of more than 360 hours of observation with the Hershel, SPIRE and PACS, from 100 um and 500. &amp;lt;p&amp;gt; North and South GOODS data is available for 100 and 160 microns (using PACS) but only the northern field is available at 250, 350 and 500 microns (using SPIRE). &amp;lt;p&amp;gt; Note that the scale and resolution of the underlying pixels is different in each band.",https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=goodsherschel1&amp;,https://skyview.gsfc.nasa.gov,NASA/GSFC HEASARC,simpleimageaccess
1,infrared,GOODSHerschel2,ivo://nasa.heasarc/skyview/goodsherschel2,"GOODS-Herschel is an open time key program of more than 360 hours of observation with the Hershel, SPIRE and PACS, from 100 um and 500. &amp;lt;p&amp;gt; North and South GOODS data is available for 100 and 160 microns (using PACS) but only the northern field is available at 250, 350 and 500 microns (using SPIRE). &amp;lt;p&amp;gt; Note that the scale and resolution of the underlying pixels is different in each band.",https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=goodsherschel2&amp;,https://skyview.gsfc.nasa.gov,NASA/GSFC HEASARC,simpleimageaccess
2,infrared,GOODSHerschel3,ivo://nasa.heasarc/skyview/goodsherschel3,"GOODS-Herschel is an open time key program of more than 360 hours of observation with the Hershel, SPIRE and PACS, from 100 um and 500. &amp;lt;p&amp;gt; North and South GOODS data is available for 100 and 160 microns (using PACS) but only the northern field is available at 250, 350 and 500 microns (using SPIRE). &amp;lt;p&amp;gt; Note that the scale and resolution of the underlying pixels is different in each band.",https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=goodsherschel3&amp;,https://skyview.gsfc.nasa.gov,NASA/GSFC HEASARC,simpleimageaccess
3,infrared,GOODSHerschel4,ivo://nasa.heasarc/skyview/goodsherschel4,"GOODS-Herschel is an open time key program of more than 360 hours of observation with the Hershel, SPIRE and PACS, from 100 um and 500. &amp;lt;p&amp;gt; North and South GOODS data is available for 100 and 160 microns (using PACS) but only the northern field is available at 250, 350 and 500 microns (using SPIRE). &amp;lt;p&amp;gt; Note that the scale and resolution of the underlying pixels is different in each band.",https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=goodsherschel4&amp;,https://skyview.gsfc.nasa.gov,NASA/GSFC HEASARC,simpleimageaccess
4,x-ray,RXTE,ivo://nasa.heasarc/skyview/rxte,"Rossi X-ray Timing Explorer was launched at the end of 1995 and up to now (2004) it has been successfully operating for more than 7 years. The mission was primarily designed to study the variability of X-ray sources on time scales from sub-milliseconds to years. The maneuvering capability of the satellite combined with the high photon throughput of its main detector (PCA) and high quality of background prediction (thanks to PCA intrumental group of LHEA, GSFC) has also made it possible to construct maps of the sky in energy band 3-20 keV. During its life time RXTE/PCA has collected a large amount of data from slew observations covering almost the entire sky. &lt;p&gt; We have utilized the slew parts of all RXTE/PCA observations performed from April 15, 1996-July 16, 2002 which amounts in total to approximately 50,000 observations. The exposure time at a given point in the map is typically between 200-500 seconds. The observational period before April 15, 1996 (High Voltage Epochs 1 and 2) was excluded from the analysis because during that time the PCA had significantly different gain and dependence of the effective area on energy. The data reduction was done using standard tools of the LHEASOFT with a set of packages written by M. Revnivtsev (HEAD/IKI, Moscow; MPA, Garching).&lt;p&gt; &lt;p&gt; The survey has sever

In [7]:
heasarc_tap_services = Registry.query( service_type='tap', publisher='heasarc')

heasarc_tap_services.show_in_notebook()

idx,waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
0,,HEASARC,ivo://nasa.heasarc/services/xamin,The HEASARC is NASA domain archive for high-energy and microwave astronomy. The Xamin interface provides access to over 600 observation and object tables. This includes observation tables for more than 30 missions and observatories and hundreds of derived object tables. Non-high energy tables are included to make it easier for users to compare information.,https://heasarc.gsfc.nasa.gov/xamin/vo/tap,https://heasarc.gsfc.nasa.gov,NASA/HEASARC,tableaccess


In [26]:
heasarc_ssa_services = Registry.query( service_type='spectr', publisher='heasarc')

heasarc_ssa_services.show_in_notebook()

idx,waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
0,gamma-ray#x-ray,INTEGRAL/BSC,ivo://nasa.heasarc/intbsc,"The INTEGRAL Bright Source Catalog is based on publicly available data from the two main instruments (IBIS and SPI) on board INTEGRAL (see Winkler et al. 2003, A&amp;amp;A, 411, L1 for a description of the INTEGRAL spacecraft and instrument packages). INTEGRAL began collecting data in October 2002. This catalog will be regularly updated as data become public (~14 months after they are obtained). The Bright Source Catalog is a collaborative effort between the INTEGRAL Science Data Center (ISDC) in Switzerland and the NASA Goddard Space Flight Center (GSFC) INTEGRAL Guest Observer Facility (GOF). The results presented here are a result of a semi-automated analysis and they should be considered as approximate: they are intended to serve as a guideline to those interested in pursuing more detailed follow-up analyses. The data from the imager ISGRI (Lebrun et al. 2003, A&amp;amp;A, 411, L141) have been analyzed at the INTEGRAL Science Data Centre (ISDC), while the SPI (Vedrenne et al. 2003, A&amp;amp;A, 411, L63) data analysis was performed at GSFC as a service of the INTEGRAL GOF.",https://heasarc.gsfc.nasa.gov/xamin/vo/ssa?table=intbsc&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/intbsc.html,NASA/GSFC HEASARC,simplespectralaccess
1,gamma-ray,SwiftXRT,ivo://nasa.heasarc/swiftxrlog,"The XRT runs only one type of configuration mode/window in a given time interval. The table therefore contains for a given time interval a single record that describes one configuration. A new record is generated when the following is changing within an observation: new operating mode , new pointing mode, or new window configuration.",https://heasarc.gsfc.nasa.gov/xamin/vo/ssa?table=swiftxrlog&amp;,https://heasarc.gsfc.nasa.gov/W3Browse/all/swiftxrlog.html,NASA/GSFC HEASARC,simplespectralaccess
2,x-ray,Chandra,ivo://nasa.heasarc/chanmaster,"This database table contains all of the observations made by the Chandra X-Ray Observatory (CXO, formerly known as the Advanced X-ray Astrophysics Facility or AXAF) as part of the Performance Verification and Calibration (PVC) phase and also contains all of the subsequent Cycles' Guaranteed Time Observers (GTO) and General Observer (GO) targets, and any Director's Discretionary Time (DDT) targets that have been observed. It also includes scheduled and as-yet-not-scheduled targets. The HEASARC updates this database table on a twice-weekly basis by querying the database table at the Chandra X-Ray Center (CXC) website, as discussed in the Provenance section. For observations whose status is 'archived', data products can be retrieved from the HEASARC's mirror of the CXC's Chandra Data Archive (CDA). The CXC should be acknowledged as the source of Chandra data. The PVC phase was during the first few months of the CXO mission; some of the calibration observations that are for monitoring purposes will be performed in later mission cycles. All calibration data (entries with Type = CAL in this database) are placed immediately into the CXO public data archive at the Chandra X-Ray Observatory Center (CXC); please see the Web page at &amp;lt;a href=""http://asc.harvard.edu/""&amp;gt;http://asc.harvard.edu/&amp;lt;/a&amp;gt; for more information on the CXC data archive). GTO observations during Cycle 1 or any subsequent Cycle will probably occupy 100% of months 3-4, 30% of months 5-22, and 15% of the available time for the remainder of the mission. Guaranteed Time Observers will have the same proprietary data rights as General Observers (i.e., their data will be placed in the public CXC archive 12 months after they have received the data in usable form). For detailed information on the Chandra Observatory and datasets see: &amp;lt;pre&amp;gt; &amp;lt;a href=""http://cxc.harvard.edu/""&amp;gt;http://cxc.harvard.edu/&amp;lt;/a&amp;gt; for general Chandra information &amp;lt;a hre

In [36]:
ipac_cone_services = Registry.query( service_type='cone', publisher='nasa/ipac')
ipac_image_services = Registry.query( service_type='image', publisher='nasa/ipac')
ipac_tap_services = Registry.query( service_type='table', publisher='nasa/ipac')
ipac_ssa_services = Registry.query( service_type='spectr', publisher='nasa/ipac')

print(f'IPAC Cone: {len(ipac_cone_services)}')
print(f'IPAC Image: {len(ipac_image_services)}')
print(f'IPAC TAP: {len(ipac_tap_services)}')
print(f'IPAC SSA: {len(ipac_ssa_services)}')


IPAC Cone: 380
IPAC Image: 84
IPAC TAP: 1
IPAC SSA: 2


In [44]:
stsci_cone_services = Registry.query( service_type='cone', publisher='Space Telescope Science Institute')
stsci_image_services = Registry.query( service_type='image', publisher='Space Telescope Science Institute')
stsci_tap_services = Registry.query( service_type='table', publisher='Space Telescope Science Institute')
stsci_ssa_services = Registry.query( service_type='spectr', publisher='Space Telescope Science Institute')

print(f'STScI Cone: {len(stsci_cone_services)}')
print(f'STScI Image: {len(stsci_image_services)}')
print(f'STScI TAP: {len(stsci_tap_services)}')
print(f'STScI SSA: {len(stsci_ssa_services)}')

STScI Cone: 63
STScI Image: 41
STScI TAP: 4
STScI SSA: 14
